# Imports

In [1]:
import numpy as np

from scipy.stats import skew, kurtosis
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from src.datasets.supervised_dataset import SupervisedDataset
from src.models.label_encoder import LabelEncoder

# Load Data

In [2]:
train_path = 'data/supervised/train'
val_path = 'data/supervised/val'

features = ['accelerometerAccelerationX(G)', 
            'accelerometerAccelerationY(G)',
            'accelerometerAccelerationZ(G)', 
            'motionYaw(rad)', 
            'motionRoll(rad)',
            'motionPitch(rad)', 
            'motionRotationRateX(rad/s)',
            'motionRotationRateY(rad/s)', 
            'motionRotationRateZ(rad/s)',
            'motionUserAccelerationX(G)', 
            'motionUserAccelerationY(G)',
            'motionUserAccelerationZ(G)', 
            'motionQuaternionX(R)',
            'motionQuaternionY(R)', 
            'motionQuaternionZ(R)', 
            'motionQuaternionW(R)',
            'motionGravityX(G)', 
            'motionGravityY(G)', 
            'motionGravityZ(G)'
]

label_encoder = LabelEncoder()
train_data = SupervisedDataset(train_path, features, label_encoder, normalize=False)
val_data = SupervisedDataset(val_path, features, label_encoder, normalize=False)

# Compute Features

In [3]:
def compute_features(dataset):
    X = []
    y = []
    
    for sequence, label in dataset:
        # Compute the statistics for each column
        _min = np.min(sequence.numpy(), axis=0)
        _max = np.max(sequence.numpy(), axis=0)
        _std = np.std(sequence.numpy(), axis=0)
        _mean = np.mean(sequence.numpy(), axis=0)
        _skew = skew(sequence.numpy(), axis=0)
        _kurtosis = kurtosis(sequence.numpy(), axis=0)
        feats = np.concatenate((_min, _max, _std, _mean, _skew, _kurtosis)).reshape(1, -1)
        X.append(feats)
        y.append(label.item())
    
    return np.concatenate(X), np.array(y)

In [4]:
X_train, y_train = compute_features(train_data)
X_val, y_val = compute_features(val_data)

# KNN

In [26]:
k = 5
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_val)
f1 = f1_score(y_val, y_pred, average = 'macro')
print('F1-Score: {:.2f}'.format(f1))

F1-Score: 0.42


# Random Forest

In [27]:
n_estimators = 141
max_depth = 16
seed = 42

random_forest = RandomForestClassifier(n_estimators=n_estimators, 
                                    max_depth=max_depth, 
                                    random_state=seed)
random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_val)
f1 = f1_score(y_val, y_pred, average = 'macro')
print('F1-Score: {:.2f}'.format(f1))

F1-Score: 0.54


# SVM

In [28]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

svc = make_pipeline(StandardScaler(), SVC(gamma='auto'))
svc.fit(X_train, y_train)

y_pred = svc.predict(X_val)
f1 = f1_score(y_val, y_pred, average = 'macro')
print('F1-Score: {:.2f}'.format(f1))

F1-Score: 0.54
